# Case Study: Swine lagoons

***Caitlin Haedrich, Pratikshya Regmi, Anna Petrasova and Helena Mitasova***

*Center for Geospatial Analytics at NC State University*

North Carolina is one of the top pork producing states in the US, surpassed only by Minnesota and Iowa. The large industry in North Carolina consists of hundreds of large-scale farms raise pigs, processing facilities, trucks that transport the animals and fields that grow the grains for feed. Raising over 8 million pigs in a concentrated area creates one big issue: waste.

<img src="https://raw.githubusercontent.com/chaedri/chaedri.github.io/refs/heads/master/images/CAFOs.png" />

The waste is typically stored in large retention ponds referred to as *lagoons*. The waste anaerobically digests and then is spread on the nearby fields as fertilizer. However, during catastorphic flooding events such as [Hurricane Florence in 2018](https://www.npr.org/2018/09/22/650698240/hurricane-s-aftermath-floods-hog-lagoons-in-north-carolina), the flood waters can overtop the sides of the lagoon introducing the waste to the surrounding environment.

<img src="https://modernfarmer.com/wp-content/uploads/2022/02/16442235689_6f9667cc05_k-768x489.jpg" />


Part of the Cape Fear Watershed, Stocking Head Creek flows through a heavily agricultural area and has some of the highest densities of swine farms in the country. We'll compute where the swine waste would flow in the case of a storage lagoon leak and how much the creek would need to flood in order to innundate the lagoon and introduce the waste matter to the environment.

Let's use the lagoon locations and DEM to answer 4 questions:

[1. If the lagoons overflowed, what path would the waste travel to the nearest waterway?](#drain)

[2. If the stream water level rose 1 meter, would any of the lagoons be breached?](#hand)

[3. What is the upstream contributing area for a hypothetical sample point?](#upstream)

[4. What are the overland flow dynamics during a heavy rainstorm in the upstream contributing area?](#simwe)

***

## Imports and Start GRASS

Import the Python standard libraries we need.

In [ ]:
import subprocess
import sys
from pathlib import Path

sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True).strip()
)

# Import the GRASS GIS packages we need.
import grass.script as gs
import grass.jupyter as gj

In [ ]:
gj.init("./csdms-grass-2025/PERMANENT");

In [ ]:
!g.list type=all

In [ ]:
!g.region -p

<a name="drain"></a>

***

## __Question 1:__ *If the lagoons overflowed or were breached, what path would the waste travel to the nearest waterway?*

(This does happen and has serious consequences:  [news article](https://www.newsobserver.com/news/state/north-carolina/article264779224.html)).

The [r.watershed](https://grass.osgeo.org/grass-devel/manuals/r.watershed.html) tool is a popular and powerful tool for hydrology. Check out all of the outputs it can compute in the [manual page](https://grass.osgeo.org/grass-devel/manuals/r.watershed.html). Here we'll use it to compute the flow accumulation (how many cells are upstream of a given cell) and drainage direction (what direction a particle would flow from each cell). By default the tool uses multiple flow direction algorithm, which works better on a flat landscape. We don't need to fill sinks, because r.watershed uses least-cost path approach for routing through depressions. Then, we'll use [r.path](https://grass.osgeo.org/grass-devel/manuals/r.path.html) to trace the route of the waste being transported downhill from the lagoon.

In [ ]:
gs.run_command("r.watershed", elevation="elevation", accumulation="accumulation", drainage="drainage")

In [ ]:
map = gj.InteractiveMap()
map.add_raster("accumulation", opacity=0.5)
map.show()

In [ ]:
gs.run_command("r.path", input="drainage", vector_path="drain", start_points="lagoon_points")

Let's see what is the landcover the water from lagoons would flow through:

In [ ]:
map = gj.Map()
map.d_shade(color="ortho", shade="relief", brighten=50)
map.d_vect(map="drain", width=1, color="blue")
map.show()

<a name="hand"></a>

***

## __Question 2:__ *If the stream water level rose 1 meter, would any of the lagoon be breached?*

To answer this question, we'll use the HAND (height above nearest drainage) method to model the flood [(Nobre et al., 2011)](https://www.sciencedirect.com/science/article/pii/S0022169411002599).

First, we'll add the two extensions we need for this workflow.

In [ ]:
gs.run_command("g.extension", extension="r.hand")

To create a timeseries of the innundation, we can use the t flag.

In [ ]:
gs.run_command("r.hand",
               flags="t",
               elevation="elevation",
               hand="hand",
               inundation_strds="inundation"
               start_water_level=0,
               end_water_level=5,
               water_level_step=1
              )


In [ ]:
map = gj.TimeSeriesMap()
map.d_rast(map="relief")
map.d_vect(map="lagoons", fill_color="none")
map.add_raster_series("inundation")
map.show()

It looks like one lagoon would be breached and a few more are very close to flooding.

<a name="upstream"></a>

***

## __Question 3:__ *What is the upstream contributing area for a hypothetical sample point?*

To do this, we will extract a coordinate from a section of stream and then use [r.water.outlet](https://grass.osgeo.org/grass-devel/manuals/r.water.outlet.html) with the drainage direction raster to compute the upstream contribute area.

In [ ]:
gs.run_command("v.extract", input="streams", output="stream_points", type="point", where="x IS NOT NULL")

In [ ]:
map = gj.Map()
map.d_rast(map="relief")
map.d_vect(map="streams", color="blue")
map.d_vect(map="stream_points", display="cat", label_color="white", label_size=10)
map.show()

Let's use point with category 15. Vector attributes are stored in a SQL database inside the project. We use [v.to.db](https://grass.osgeo.org/grass-devel/manuals/v.to.db.html) to add the feature coordinates to the attribute table, then [v.db.select](https://grass.osgeo.org/grass-devel/manuals/v.db.select.html) to select the category and coordinates and show them as a Pandas dataframe.

In [ ]:
import pandas as pd

gs.run_command("v.to.db", map="stream_points", option="coor", type="point", columns="x,y")
df = pd.DataFrame(gs.parse_command("v.db.select", map="stream_points", columns="cat,x,y", format="json")["records"])
df

In [ ]:
[df.loc[8, 'x'], df.loc[8, 'y']]

Finally, use [r.water.outlet](https://grass.osgeo.org/grass-devel/manuals/r.water.outlet.html) to compute the upstream contributing area.

In [ ]:
gs.run_command("r.water.outlet", input="direction", output="basin_15", coordinates=[df.loc[8, 'x'], df.loc[8, 'y']])
map = gj.Map()
map.d_shade(color="basin_15", shade="relief", brighten=50)
map.show()

TODO: add viewshed here from the coordinates so we can use it on TL

<a name="simwe"></a>

***

## __Question 4:__ *What are the overland flow dynamics during a heavy rainstorm in basin 15?*

We're going to use [r.sim.water](https://grass.osgeo.org/grass-devel/manuals/r.sim.water.html) to model overland flow. The `r.sim.water` tool is the GRASS implementation of the SIMWE model ([Mitas and Mitasova, 1998](https://agupubs.onlinelibrary.wiley.com/doi/abs/10.1029/97WR03347)), a monte carlo path-tracing approach to solving the St. Venant equations for overland flow.

First, let's set the computational region to the extent of `basin_15` and a lower resolution (6 meters instead of 3) for faster compute times.

In [ ]:
gs.run_command("g.region", zoom="basin_13", res=6)

Apply a mask over the areas outside `basin_15`. Now, only areas inside `basin 15` will be displayed or used in any computation. Unlike computational region, [r.mask](https://grass.osgeo.org/grass-devel/manuals/r.mask.html) can have boundaries that are not rectangular.

In [ ]:
gs.run_command("r.mask", raster="basin_15")

Run `r.sim.water` after computing the x and y direction derivatives. We'll run a 30-minute rainstorm using the default rainfall rate of 50 mm/hr. The output will be a series a map showing water depth at each minute.

In [ ]:
gs.run_command('r.slope.aspect', elevation="elevation", dx="dx", dy="dy")
gs.run_command('r.sim.water', elevation="elevation", dx="dx", dy="dy", depth="depth", flags="t", niterations=30)

Finally, we'll create a temporal dataset and register the output depth maps. GRASS has [a library of tools](https://grass.osgeo.org/grass-devel/manuals/temporalintro.html) for temporal analyses but here, we will just create an animation of the timeseries.

In [ ]:
# Create a time series
gs.run_command("t.create",
               output="depth",
               temporaltype="relative",
               title="Overland flow depth",
               description="Overland flow depth")

# Register the time series
maps = gs.list_strings(type="raster", pattern="depth*")
gs.run_command("t.register", input="depth", maps=maps)

In [ ]:
flow_map = gj.TimeSeriesMap()
flow_map.add_raster_series("depth")
flow_map.show()

Remove the mask and reset the computational region to the original region.

In [ ]:
# Remove mask
gs.run_command("r.mask", flags="r")
# Re-set region
gs.run_command("g.region", n=131934, s=126825, w=702726, e=708443, res=3, flags="a")